In [3]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


from google.colab import auth
auth.authenticate_user()
print('Authenticated')

In [2]:
project_id = 'project-database-437915'

from google.cloud import bigquery

client = bigquery.Client(project=project_id)

**Subqueries/Nested Queries**

In [6]:
df = client.query('''
  SELECT
  `hash` AS transaction_hash,
  block_hash
FROM
  `bigquery-public-data.crypto_bitcoin.transactions`
WHERE
  TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-08-06")
  AND block_hash IN (
  SELECT
    `hash`
  FROM
    `bigquery-public-data.crypto_bitcoin.blocks`
  WHERE
    TIMESTAMP_TRUNC(timestamp, DAY) = TIMESTAMP("2024-08-06"))
LIMIT
  5''').to_dataframe()

df.head()

,transaction_hash,block_hash


Joins

In [7]:
df = client.query('''
SELECT
  t.transaction_hash,
  t.block_hash,
  tb.transaction_count
FROM (
  SELECT
    `hash` AS transaction_hash,
    block_hash
  FROM
    `bigquery-public-data.crypto_bitcoin.transactions`
  WHERE
    TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-08-06")) AS t
INNER JOIN (
  SELECT
    `hash` AS block_hash,
    transaction_count
  FROM
    `bigquery-public-data.crypto_bitcoin.blocks`
  WHERE
    TIMESTAMP_TRUNC(timestamp, DAY) = TIMESTAMP("2024-08-06"))
    AS tb
ON
  t.block_hash = tb.block_hash
ORDER BY
  t.transaction_hash
LIMIT
  5''').to_dataframe()

df.head()

,transaction_hash,block_hash,transaction_count
0,000001fec2219179a2af30fdfb0fff74abfac548189587...,000000000000000000011d43b25b77baebdfb6c9b2489b...,6328
1,00000290b427544fd20d77157511c993ec502db849d2d4...,00000000000000000002877f4d9341ecc925cf5810a943...,2480
2,000003423de3da3447ee22e73b81904afacb43c0276416...,00000000000000000002f04be8a637c3f0dfb87dd76283...,2945
3,000003a4ebb5f100c4f58e37a66b76806c33bf74abd6f5...,0000000000000000000172f222ec984cedba45fb2ef8f7...,3431
4,0000063a3200d6a576bfe3ab886c8d5ed90a17a1e54d5a...,00000000000000000002f6dc0e16bf93479238c3b39c11...,2477


Unions

In [8]:
df = client.query('''
SELECT
  `hash` AS block_hash,
  `timestamp` AS block_timestamp,
  `number` AS block_number
FROM
  `bigquery-public-data.crypto_bitcoin.blocks`
WHERE
  TIMESTAMP_TRUNC(timestamp, DAY) = TIMESTAMP("2024-08-06")

UNION DISTINCT

SELECT
  block_hash,
  block_timestamp,
  block_number
FROM
  `bigquery-public-data.crypto_bitcoin.transactions`
WHERE
  TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-08-06")
LIMIT 5''').to_dataframe()

df.head()

,block_hash,block_timestamp,block_number
0,00000000000000000000fe7082d894d19baeb3826bbb1d...,2024-08-06 03:34:24+00:00,855590
1,0000000000000000000154f2660340d562e57341fab368...,2024-08-06 03:22:57+00:00,855588
2,000000000000000000020d03a0ad0421b97ef97a61de9f...,2024-08-06 23:16:44+00:00,855706
3,0000000000000000000312ef55c893477519a25e6a0c1a...,2024-08-06 01:58:20+00:00,855579
4,00000000000000000000c9f507b37c17b88733473b7ae7...,2024-08-06 14:45:09+00:00,855659


Aggregate Functions

In [9]:
df = client.query('''
SELECT
  block_hash,
  COUNT(`hash`) AS total_transaction
FROM
  `bigquery-public-data.crypto_bitcoin.transactions`
WHERE
  TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-08-06")
  AND block_hash IN (
    SELECT
      `hash`
    FROM
      `bigquery-public-data.crypto_bitcoin.blocks`
    WHERE
      TIMESTAMP_TRUNC(timestamp, DAY) = TIMESTAMP("2024-08-06"))
GROUP BY
  block_hash
Limit 5''').to_dataframe()

df.head()

,block_hash,total_transaction
0,000000000000000000009167609a340ecbb6fa81f90a10...,1846
1,0000000000000000000312ef55c893477519a25e6a0c1a...,3004
2,0000000000000000000071723b8087e66e80944bc7798a...,4533
3,000000000000000000022a026fa28e910a716488d26764...,5816
4,000000000000000000000cbaa07d62042484c49ffce543...,5646


Window Functions

In [10]:
df = client.query('''
SELECT
  block_hash,
  total_transaction,
  ROW_NUMBER() OVER (ORDER BY total_transaction DESC) AS rank
FROM (
  SELECT
    block_hash,
    COUNT(`hash`) AS total_transaction
  FROM
    `bigquery-public-data.crypto_bitcoin.transactions`
  WHERE
    TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-08-06")
    AND block_hash IN (
      SELECT
        `hash`
      FROM
        `bigquery-public-data.crypto_bitcoin.blocks`
      WHERE
        TIMESTAMP_TRUNC(timestamp, DAY) = TIMESTAMP("2024-08-06"))
  GROUP BY
    block_hash
) AS transaction_counts
ORDER BY rank
Limit 5''').to_dataframe()

df.head()


,block_hash,total_transaction,rank
0,00000000000000000001cf7fd65d7cac4bab533ca3bcfb...,7855,1
1,000000000000000000020686ab68363ebaadd5d87c9cf1...,7402,2
2,000000000000000000018028ae422d7b637ec8c41ff094...,7239,3
3,0000000000000000000053d7f74913d44db8f74727b321...,6836,4
4,000000000000000000021fcc083737c58b717699815348...,6828,5


Common Table Expressions (CTEs)

In [11]:
df = client.query('''
WITH
  transaction_block AS (
    SELECT
      `hash` AS transaction_hash,
      block_hash
    FROM
      `bigquery-public-data.crypto_bitcoin.transactions`
    WHERE
      TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-08-06")
  ),
  block_parent AS (
    SELECT
      `hash` AS block_hash,
      version
    FROM
      `bigquery-public-data.crypto_bitcoin.blocks`
    WHERE
      TIMESTAMP_TRUNC(timestamp, DAY) = TIMESTAMP("2024-08-06")
      AND version = 541065216
  )
SELECT
  transaction_block.transaction_hash,
  transaction_block.block_hash,
  block_parent.version
FROM
  transaction_block
INNER JOIN
  block_parent
ON
  transaction_block.block_hash = block_parent.block_hash
ORDER BY
  transaction_block.transaction_hash
LIMIT
  5''').to_dataframe()

df.head()

,transaction_hash,block_hash,version
0,00125a1303ce36674c004c04f777672f4595e05d50bdca...,000000000000000000000c8d692c133d531cd99b6de92b...,541065216
1,00200a88df23ce0e5176709116386367bfc90f685c4256...,000000000000000000000c8d692c133d531cd99b6de92b...,541065216
2,002f9bdcf71b8d0c1c3fc6edb915c4fbd142885c9fff58...,000000000000000000000c8d692c133d531cd99b6de92b...,541065216
3,003a33603e8a60183d157ead4fe7e3ab52495b256bbd20...,000000000000000000000c8d692c133d531cd99b6de92b...,541065216
4,00404ec59161d16b8db66c7ed92566750344f09b55a8de...,000000000000000000000c8d692c133d531cd99b6de92b...,541065216


Pivot

In [13]:
df = client.query('''
SELECT *
FROM (
  SELECT
    block_hash,
    CASE WHEN is_coinbase THEN 'Coinbase' ELSE 'Non-Coinbase' END AS transaction_type
  FROM
    `bigquery-public-data.crypto_bitcoin.transactions`
  WHERE
    TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-08-06")
) AS src
PIVOT (
  COUNT(transaction_type)
  FOR transaction_type IN ('Coinbase' AS coinbase_transactions, 'Non-Coinbase' AS non_coinbase_transactions)
) AS pvt
ORDER BY block_hash
LIMIT 5;''').to_dataframe()

df.head()

,block_hash,coinbase_transactions,non_coinbase_transactions
0,000000000000000000000c8d692c133d531cd99b6de92b...,1,4727
1,000000000000000000000cbaa07d62042484c49ffce543...,1,5645
2,000000000000000000001d17cac85bb20f8887f31ac9c2...,1,2705
3,0000000000000000000025451521cc8417a4997699301c...,1,3255
4,00000000000000000000286f3e090b3612431fe9de5a4e...,1,2967


String Manipulation

In [15]:
df = client.query('''
SELECT
  block_hash,
  SUBSTR(block_hash, 1, 10) AS short_block_hash
FROM
  `bigquery-public-data.crypto_bitcoin.transactions`
WHERE
  TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-08-06")
LIMIT
  5;
''').to_dataframe()

df.head()

,block_hash,short_block_hash
0,000000000000000000003192bfd239005a7b2b4c47bfa0...,0000000000
1,0000000000000000000117c83c75aaf2462c799590ab78...,0000000000
2,000000000000000000011854d78feaf55f65393fd6714d...,0000000000
3,00000000000000000002f04be8a637c3f0dfb87dd76283...,0000000000
4,000000000000000000018bb6b0a932d417f21dc188f029...,0000000000


Date and Time

In [16]:
df = client.query('''
SELECT
  block_hash,
  block_timestamp,
  EXTRACT(YEAR FROM block_timestamp) AS transaction_year,
  EXTRACT(MONTH FROM block_timestamp) AS transaction_month,
  EXTRACT(DAY FROM block_timestamp) AS transaction_day
FROM
  `bigquery-public-data.crypto_bitcoin.transactions`
WHERE
  TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-08-06")
LIMIT
  5;
''').to_dataframe()

df.head()

,block_hash,block_timestamp,transaction_year,transaction_month,transaction_day
0,000000000000000000003192bfd239005a7b2b4c47bfa0...,2024-08-06 00:57:37+00:00,2024,8,6
1,00000000000000000002f8fec5dd41f2b611c5997d4acc...,2024-08-06 04:45:04+00:00,2024,8,6
2,00000000000000000002f779fcd151501a8621d054255b...,2024-08-06 05:11:56+00:00,2024,8,6
3,00000000000000000001e5221802bbfc4fa07886f6167b...,2024-08-06 07:32:39+00:00,2024,8,6
4,000000000000000000018bb6b0a932d417f21dc188f029...,2024-08-06 08:29:50+00:00,2024,8,6


Case Statement

In [18]:
df = client.query('''
SELECT
  block_hash,
  size,
  CASE
    WHEN size < 200 THEN 'Small'
    WHEN size BETWEEN 200 AND 300 THEN 'Medium'
    ELSE 'Large'
  END AS transaction_size_category
FROM
  `bigquery-public-data.crypto_bitcoin.transactions`
WHERE
  TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-08-06")
LIMIT
  5;
''').to_dataframe()

df.head()

Forbidden: 403 Quota exceeded: Your project exceeded quota for free query bytes scanned. For more information, see https://cloud.google.com/bigquery/docs/troubleshoot-quotas; reason: quotaExceeded, location: unbilled.analysis, message: Quota exceeded: Your project exceeded quota for free query bytes scanned. For more information, see https://cloud.google.com/bigquery/docs/troubleshoot-quotas

Location: US
Job ID: 2213e544-79bd-44c8-8b4f-461489d75389


Recrusive Queries

In [23]:
df = client.query('''
WITH RECURSIVE block_hierarchy AS (
  SELECT
    `hash` AS block_hash,
    number,
    1 AS level
  FROM
    `bigquery-public-data.crypto_bitcoin.blocks`
  WHERE
    number = 367853

  UNION ALL

  SELECT
    b.`hash` AS block_hash,
    b.number,
    bh.level + 1
  FROM
    `bigquery-public-data.crypto_bitcoin.blocks` AS b
  INNER JOIN
    block_hierarchy AS bh ON b.number = bh.number + 1
)

SELECT
  block_hash,
  number,
  level
FROM
  block_hierarchy
ORDER BY
  level, number;

''').to_dataframe()

df.head()

KeyboardInterrupt: 

UDF


In [24]:
df = client.query('''
CREATE TEMP FUNCTION
    calculate_fee(input_value FLOAT64, output_value FLOAT64) as (
        input_value - output_value
    );

SELECT
  t.hash AS transaction_hash,
  t.input_value,
  t.output_value,
  calculate_fee(t.input_value, t.output_value) AS transaction_fee
FROM
  `bigquery-public-data.crypto_bitcoin.transactions` AS t
WHERE
  t.block_number = 367853
ORDER BY
  t.hash;
''').to_dataframe()

df.head()

Forbidden: 403 Quota exceeded: Your project exceeded quota for free query bytes scanned. For more information, see https://cloud.google.com/bigquery/docs/troubleshoot-quotas; reason: quotaExceeded, location: unbilled.analysis, message: Quota exceeded: Your project exceeded quota for free query bytes scanned. For more information, see https://cloud.google.com/bigquery/docs/troubleshoot-quotas

Location: US
Job ID: 00f0dda8-c4c8-4629-8e4d-0a9a852264ad
